In [287]:
import sqlite3
import pandas as pd
import os

# changing the "current working directory"
os.chdir('/home/ec2-user/SageMaker/')

# load csv file into pandas dataframe
# reference: https://stackoverflow.com/a/28802613 
csvfile = "resources/phase3/simfin/us-income-ttm.csv"

dfUSINCOME = pd.read_csv(csvfile, sep=';') # reference https://stackoverflow.com/a/24606473
dfUSINCOME.columns = df.columns.str.replace(' ', '') # remove spaces in column names

# instructions for how to create a database file: https://www.sqlitetutorial.net/sqlite-python/creating-database/
# let create an empty database
db_file = "simfim.sqlite" # we need a name for our db file
conn = sqlite3.connect(db_file) # sqlite will create this file for us
table_name = "tblSimFin" # we need a name for our db table

# now we have everyting we need for our reference line above.
# let's put it together

dfUSINCOME.to_sql(table_name, conn, if_exists='replace', index=False)

In [153]:
con = sqlite3.connect("resources/phase3/sp500/sp500.sqlite")

In [59]:
#Q1 Observe the years with the highest differnce between High and Low in SP500
# Quick code to see lowest open ever
dfSP500[dfSP500.Open == min(dfSP500.Open)]

,id,Date,Open,High,Low,Close,Adj Close,Volume
7166,7167,1978-07-31,100,101.18,99.37,100.68,100.68,33990000
7481,7482,1979-10-26,100,101.31,99.59,100.57,100.57,29660000


In [386]:
#Create dataframe grouped by year and find the high for the year

dfsp500yearHigh = dfSP500.groupby(['year'], as_index = False)["High"].max()

In [387]:
dfsp500yearHigh.head(10)

,year,High
0,1950,20.43
1,1951,23.85
2,1952,26.59
3,1953,26.66
4,1954,35.98
5,1955,46.41
6,1956,49.64
7,1957,49.13
8,1958,55.21
9,1959,60.71


In [361]:
#same as above except find the lowest low

dfsp500Low = dfSP500.groupby(['year'], as_index = False)["Low"].min()

In [362]:
dfsp500Low.head(10)

,year,Low
0,1950,16.66
1,1951,20.69
2,1952,23.09
3,1953,22.71
4,1954,24.8
5,1955,34.58
6,1956,43.11
7,1957,38.98
8,1958,40.33
9,1959,53.58


In [366]:
#Joining the two dataframes to create one where each year is grouped and has highest high along with lowest low


dfSP500MinMax = dfsp500yearHigh.set_index('year').join(dfsp500Low.set_index('year'))

In [367]:
dfSP500MinMax.head(10)

,High,Low
year,,
1950,20.43,16.66
1951,23.85,20.69
1952,26.59,23.09
1953,26.66,22.71
1954,35.98,24.8
1955,46.41,34.58
1956,49.64,43.11
1957,49.13,38.98
1958,55.21,40.33


In [370]:
#Converting values to numeric values for subtraction


dfSP500MinMax['High'] = pd.to_numeric(dfSP500MinMax['High'])
dfSP500MinMax['Low'] = pd.to_numeric(dfSP500MinMax['Low'])

In [371]:
dfSP500MinMax.head(10)

,High,Low
year,,
1950,20.43,16.66
1951,23.85,20.69
1952,26.59,23.09
1953,26.66,22.71
1954,35.98,24.80
1955,46.41,34.58
1956,49.64,43.11
1957,49.13,38.98
1958,55.21,40.33


In [372]:
#Create column to show difference between max High and min Low


dfSP500MinMax['Difference'] = dfSP500MinMax.High - dfSP500MinMax.Low

In [373]:
dfSP500MinMax.head(10)

,High,Low,Difference
year,,,
1950,20.43,16.66,3.77
1951,23.85,20.69,3.16
1952,26.59,23.09,3.50
1953,26.66,22.71,3.95
1954,35.98,24.80,11.18
1955,46.41,34.58,11.83
1956,49.64,43.11,6.53
1957,49.13,38.98,10.15
1958,55.21,40.33,14.88


In [376]:
#Sorting values descending order by difference.



dfSP500MinMax = dfSP500MinMax.sort_values(by = 'Difference', ascending = False)

In [377]:
#Our Final Answer
#Years ordered by difference between highest high and lowest low throughout the year
#I created a years column earlier in the database, but on my review I think I accidentally deleted the code but the dataframe did not change


dfSP500MinMax.head(10)

,High,Low,Difference
year,,,
2018,2940.91,2346.58,594.33
2016,2277.53,1810.10,467.43
2017,2694.97,2245.13,449.84
2013,1849.44,1426.19,423.25
2014,2093.55,1737.92,355.63
2000,1552.87,1254.07,298.80
2011,1370.58,1074.77,295.81
2015,2134.72,1867.01,267.71
1999,1473.10,1205.46,267.64


In [157]:
dfUSINCOME.columns

Index(['Ticker', 'SimFinId', 'Currency', 'FiscalYear', 'FiscalPeriod',
       'ReportDate', 'PublishDate', 'Shares(Basic)', 'Shares(Diluted)',
       'Revenue', 'CostofRevenue', 'GrossProfit', 'OperatingExpenses',
       'Selling,General&Administrative', 'Research&Development',
       'Depreciation&Amortization', 'OperatingIncome(Loss)',
       'Non-OperatingIncome(Loss)', 'InterestExpense,Net',
       'PretaxIncome(Loss),Adj.', 'AbnormalGains(Losses)',
       'PretaxIncome(Loss)', 'IncomeTax(Expense)Benefit,Net',
       'Income(Loss)fromContinuingOperations', 'NetExtraordinaryGains(Losses)',
       'NetIncome', 'NetIncome(Common)'],
      dtype='object')

In [160]:
#Q2
#Find Companies with 2x RD costs as SGA in 2018 sorted by Basic Shares
#Rename Research & Development as RD
dfUSINCOME = dfUSINCOME.rename(columns= {"Research&Development":"RD"})
dfUSINCOME = dfUSINCOME.rename(columns= {"Selling,General&Administrative":"SGA"})

In [187]:
#Editing DB to only have those were RD costs are 2 times SGA costs
dfRD = dfUSINCOME[dfUSINCOME.RD > (2 * dfUSINCOME.SGA)]

In [188]:
#Only wanted those in year 2018
dfRD = dfRD[dfRD.FiscalYear == 2018]

In [197]:
#Sorting by basic shares descending

dfRD = dfRD.sort_values(by='Shares(Basic)', ascending = False)

In [383]:
#Companies with RD costs 2x SGA Costs in year 2018, sorted by Basic Shares and Operating Income included
dfRD2 = dfRD[['Ticker', 'FiscalYear', 'RD', 'SGA', 'Shares(Basic)', 'OperatingIncome(Loss)']]

In [384]:
dfRD2.head(10)

,Ticker,FiscalYear,RD,SGA,Shares(Basic),OperatingIncome(Loss)
27320,MSFT,2018,-1.350500e+10,-2.065700e+10,7.725750e+09,3.032400e+10
27321,MSFT,2018,-1.394700e+10,-2.137000e+10,7.714500e+09,3.109800e+10
27322,MSFT,2018,-1.430700e+10,-2.183900e+10,7.707750e+09,3.266700e+10
27323,MSFT,2018,-1.472600e+10,-2.222300e+10,7.700000e+09,3.505800e+10
223,AAPL,2018,-1.211700e+10,-1.554600e+10,5.170796e+09,6.425900e+10
224,AAPL,2018,-1.271900e+10,-1.597800e+10,5.120568e+09,6.605600e+10
225,AAPL,2018,-1.348300e+10,-1.630300e+10,5.042337e+09,6.790000e+10
11077,CSCO,2018,-6.081000e+09,-1.109500e+10,4.993000e+09,1.234900e+10
11078,CSCO,2018,-6.122000e+09,-1.113500e+10,4.970250e+09,1.249400e+10
226,AAPL,2018,-1.423600e+10,-1.670500e+10,4.955377e+09,7.089800e+10


In [211]:
dfUSINCOME = dfUSINCOME.rename(columns= {"OperatingIncome(Loss)":"OperatingIncome"})

In [217]:
dfUSINCOME.columns

Index(['Ticker', 'SimFinId', 'Currency', 'FiscalYear', 'FiscalPeriod',
       'ReportDate', 'PublishDate', 'Shares(Basic)', 'Shares(Diluted)',
       'Revenue', 'CostofRevenue', 'GrossProfit', 'OperatingExpenses', 'SGA',
       'RD', 'Depreciation&Amortization', 'OperatingIncome',
       'Non-OperatingIncome(Loss)', 'InterestExpense,Net',
       'PretaxIncome(Loss),Adj.', 'AbnormalGains(Losses)',
       'PretaxIncome(Loss)', 'IncomeTax(Expense)Benefit,Net',
       'Income(Loss)fromContinuingOperations', 'NetExtraordinaryGains(Losses)',
       'NetIncome', 'NetIncome(Common)'],
      dtype='object')

In [228]:
dfUSINCOME[dfUSINCOME.OperatingIncome == max(dfUSINCOME.OperatingIncome)]

,Ticker,SimFinId,Currency,FiscalYear,FiscalPeriod,ReportDate,PublishDate,Shares(Basic),Shares(Diluted),Revenue,...,Non-OperatingIncome(Loss),"InterestExpense,Net","PretaxIncome(Loss),Adj.",AbnormalGains(Losses),PretaxIncome(Loss),"IncomeTax(Expense)Benefit,Net",Income(Loss)fromContinuingOperations,NetExtraordinaryGains(Losses),NetIncome,NetIncome(Common)
44627,XOM,121214,USD,2012,Q1,2012-03-31,2012-05-03,4.808000e+09,4.811250e+09,4.769670e+11,...,-5.576200e+10,-325000000.0,71855000000,NaN,71855000000,-3.076300e+10,41092000000,NaN,39860000000,39860000000


In [239]:
dfUSINCOME = dfUSINCOME.rename(columns= {"Selling,General&Administrative":"SGA"})

In [278]:
#Q3 Find the top earners by average Operating Income
#Sort Values by Operating Income

dfYear = dfUSINCOME.sort_values('OperatingIncome', ascending = False)

In [271]:
#GroupBy Ticker Symbol and find Mean Operating Income

dfTopEarners = dfYear.groupby(["Ticker"], as_index = False)["OperatingIncome"].mean()

In [273]:
dfTopEarners.head(10)

,Ticker,OperatingIncome
0,A,7.325312e+08
1,AA,1.725750e+09
2,AAC,-1.435150e+07
3,AAL,2.774485e+09
4,AAN,2.388760e+08
5,AAOI,2.466380e+07
6,AAON,6.168410e+07
7,AAP,6.923873e+08
8,AAPL,5.240324e+10
9,AAWW,5.512241e+08


In [276]:
#Sorting by Mean Operating Income


dfTopEarners = dfTopEarners.sort_values(by = 'OperatingIncome', ascending = False)

In [277]:
#Our Top 10 Companies by Mean operating Income and our Answer
#Looks like the largest mean operating income belongs to Exon Mobile
dfTopEarners.head(10)

,Ticker,OperatingIncome
1827,XOM,8.469916e+10
8,AAPL,5.240324e+10
1112,MSFT,2.868643e+10
275,BRKA,2.555033e+10
1795,WMT,2.479518e+10
1758,VZ,2.262768e+10
1582,T,2.074154e+10
912,JNJ,1.865856e+10
739,GOOG,1.848311e+10
477,CVX,1.696743e+10


In [409]:
#Q4
#What years had the largest revenu



dfRevenue = dfUSINCOME.groupby(['FiscalYear'], as_index = False)["Revenue"].max()


In [411]:
dfRevenue.sort_values(by = "Revenue", ascending = False)

,FiscalYear,Revenue
10,2018,5.144050e+11
9,2017,5.003430e+11
8,2016,4.858730e+11
6,2014,4.856510e+11
7,2015,4.856210e+11
4,2012,4.769670e+11
5,2013,4.762940e+11
3,2011,4.670290e+11
2,2010,4.218490e+11
1,2009,4.080850e+11


In [412]:
#Q5
#What Fiscal Quarter shows the highest Average Revenues
#Decided on asking the question of who has the highest sum of Net Extraordinary Gains

#This was to find max of Operating Expense by Fiscal Quarter
dfAvgRev = dfUSINCOME.groupby(['FiscalPeriod'], as_index = False)["OperatingExpenses"].max()



In [414]:
dfAvgRev.head(10)

,FiscalPeriod,OperatingExpenses
0,Q1,2.171600e+09
1,Q2,5.311300e+09
2,Q3,8.451800e+09
3,Q4,1.110132e+09


In [434]:
#an attempt

dfTicker = dfUSINCOME[["Ticker", "NetExtraordinaryGains(Losses)"]]

In [436]:
dfTicker.head(5)

,Ticker,NetExtraordinaryGains(Losses)
0,A,NaN
1,A,NaN
2,A,NaN
3,A,NaN
4,A,NaN


In [446]:
#Group by Ticker symbol and show sum of Net Extraordinary Gains



dfGroup = dfTicker.groupby(['Ticker'], as_index = False)["NetExtraordinaryGains(Losses)"].sum()

In [471]:
#Renamed Column to Net Gains

dfGroup = dfGroup.rename(columns = {"NetExtraordinaryGains(Losses)":"NetGains"})

In [472]:
#Filter out all gains less than 0
dfGroup = dfGroup[dfGroup.NetGains > 0]

In [475]:
#Sort by NetGains descending


dfGroup = dfGroup.sort_values(by = 'NetGains', ascending = False)

In [476]:
#Our answer to top companies by sum of Net Gains

dfGroup.head(10)

,Ticker,NetGains
63,AGN,8.320550e+10
1279,PG,2.435700e+10
539,DVN,1.984500e+10
20,ABT,1.655700e+10
591,EQR,1.654295e+10
1719,UTX,1.614500e+10
1576,SYMC,1.329800e+10
1248,OXY,1.267600e+10
212,BAX,9.859000e+09
1307,PPG,9.492000e+09
